# Traffic Sign Classification Challenge 2022
Author: Agnieszka Mikołajczyk

Wyzwanie polega na klasyfikacji znaków drogowych przy pomocy sieci neuronowych. Udostępniona baza danych zawiera po 800 obrazów.

Są cztery klasy:


*   znak stopu,
*    przejście dla pieszych,
*    sygnalizacja świetlna,
*   limit prędkości.



![traffic sign example images](https://image.slidesharecdn.com/2-sign-signals-and-markings1-171206152608/95/signs-signals-and-markings-1-638.jpg?cb=1512574003)

### Wskazówki

* Aby ułatwić sobie ocenę jakości danej sieci neuronowej warto wydzielić ze zbioru treningowego zbiór walidacyjny, na którym będą mogli państwo oceniać zdolności generalizacyjne sieci.
* W przypadku małych sieci neuronowych warto roważyć zmniejszenie obrazów przed treningiem
* W przypadku klasyfikacji wieloklasowej warto oceniać wyniki korzystając z "confusion matrix". Po wyświetleniu jej widać, które klasy są mylone z którymi. Po przekątne chcemy mieć jak najwyższe wyniki.
![confusion matrix](https://dawn.cs.stanford.edu/assets/img/2017-08-11-tanda/data_aug_basic.png)
* można włączyć przyspieszenie GPU. Trzeba wejść w: Kernel -> Change Kernel -> GPU. Spowoduje to kilkunastrokrotne przyspieszenie obliczeń!!
![instruction how to set gpu](https://miro.medium.com/max/740/1*WNovJnpGMOys8Rv7YIsZzA.png)
* Warto rozważyć kodowanie wyjść typu one-hot encoding. Zazwyczaj jest łatwiejsze w treningu. W przypadku klasyfikacji 10 cyfr jedynke zakodowalibyśmy jako: 1 na pozycji drugiej (na pierwszej pozycji jest 0), a reszta 0.

```
[0 1 0 0 0 0 0 0 0 0] -> 1
[0 0 1 0 0 0 0 0 0 0] -> 2
[0 0 0 0 0 0 0 0 0 1] -> 9
```
![one hot encoding example](https://i.imgur.com/mtimFxh.png)

In [ ]:
!pip install pytorch-lightning

Now, go to your kaggle account and download dataset. Upload it below.

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!unzip -q traffic-sign-2022.zip #uzipping dataset

Now we will seed everything so the results are reproducible

In [ ]:
import random
import numpy as np
import torch
import os
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(2021)

## Loading data
Now, we will preprocess and load training set

In [ ]:
NUM_EPOCHS = 20 # number of times which the entire dataset is passed throughout the model
BATCH_SIZE = 64 # the size of input data took for one iteration
lr = 1e-3 # size of step
INPUT_SIZE = 28

In [ ]:
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

transform = transforms.Compose([
        transforms.Resize([INPUT_SIZE,INPUT_SIZE]),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4588, 0.5020, 0.5196],
                             std=[0.1665, 0.1690, 0.1644])
    ])

train_dataset = dsets.ImageFolder(
        root='data/train',
        transform=transform,
    )

train_size = round(len(train_dataset) * 0.6)
valid_size = len(train_dataset) - train_size
train_set, valid_set = torch.utils.data.random_split(train_dataset,
                                                     [train_size, valid_size])

train_gen = torch.utils.data.DataLoader(dataset=train_set,
                                      batch_size = BATCH_SIZE,
                                      shuffle = True)

valid_gen = torch.utils.data.DataLoader(dataset = valid_set,
                                      batch_size = valid_size, 
                                      shuffle = False)


print('Loaded ', len(train_set), 'training images and ', len(valid_set), ' validation images.')
print('Classes detected: ', train_dataset.classes)

In [ ]:
import matplotlib.pyplot as plt
# Show example images after transforms
data_iter = iter(train_gen)
images, labels = next(data_iter)
fig, axes = plt.subplots(figsize = (18,10), ncols=8)
for ii in range(8):
    ax = axes[ii]
    ax.imshow(images[ii].permute(1,2,0))

## Architecture and training
Defining simple net - fully connected network with 2 layers.

In [ ]:
import pytorch_lightning as pl
import pandas as pd
import seaborn as sn
from sklearn.metrics import confusion_matrix, classification_report
import timm

class ConvNetwork(pl.LightningModule):
    def __init__(self):
      super(ConvNetwork,self).__init__()
      self.model = timm.create_model('tinynet_e', pretrained=True)
      self.fc = nn.Linear(32*20*20, 4) # output layer

      self.loss_function = nn.CrossEntropyLoss()

    def forward(self,x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = x.view(-1, 32*20*20)
        out = self.fc(x)
        return out
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=lr)
    
    def training_step(self, batch, batch_idx):
        images,labels = batch
        outputs = self(images)
        loss = self.loss_function(outputs, labels)
        self.log('loss', loss) # add loss to track
        return loss

## Training

In [ ]:
net = ConvNetwork()
trainer = pl.Trainer(max_epochs=NUM_EPOCHS,log_every_n_steps=1)
trainer.fit(net,train_gen, valid_gen)

## Testing neural network

In [ ]:
import pandas as pd
import seaborn as sn
from sklearn.metrics import confusion_matrix, classification_report

def generate_preds(valid_gen):
    with torch.no_grad():
        for i, (images,labels) in enumerate(valid_gen):
            outputs = net(images)
            _, preds = torch.max(outputs, 1)
    return preds, labels

def plot_confusion_matrix(confusion_matrix_stats, target_names):
    df_cm = pd.DataFrame(confusion_matrix_stats, target_names, target_names)
    plt.figure(figsize=(10,7)) # create empty figure to draw on
    sn.set(font_scale=1.2) # for label size
    sn.heatmap(df_cm, annot=True, annot_kws={"size": 14}) # font size
    plt.show()

preds, labels = generate_preds(valid_gen)
confusion_matrix_stats = confusion_matrix(labels, preds)
plot_confusion_matrix(confusion_matrix_stats, target_names = train_dataset.classes)



In [ ]:
print(" +=============+ Classification report +=============+ \n")
print(classification_report(labels, preds,  target_names = train_dataset.classes))